In [1]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

### Tabla detalle sucursal

In [3]:
dir_archivo = "/home/tonatiuh/Documents/Desarrollo/ZophiaLearning/ejercicios/"
nombre_archivo = 'ecommerce/curated/detalle_sucursal.parquet'

In [4]:
df = spark.read.format("parquet")\
        .load(dir_archivo+nombre_archivo)

In [5]:
df.select('sucursal').show(n=5, truncate=False)

+--------+
|sucursal|
+--------+
|AGS-SVu |
|SON-aHW |
|SLP-lxZ |
|COL-ojJ |
|VER-Bun |
+--------+
only showing top 5 rows



#### Direccion

In [6]:
df_procesado = df.withColumn('sucursal_split', F.split(F.col('sucursal'), '-'))
df_procesado = df_procesado.withColumn('estado', F.col('sucursal_split').getItem(0))

In [7]:
df_estado = df_procesado.drop('sucursal_split')
df_estado.cache()

DataFrame[sucursal_id: string, sucursal: string, direccion: string, telefono: string, estado: string]

In [8]:
df_estado.show(n=5, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 sucursal_id | C-72pY                                        
 sucursal    | AGS-SVu                                       
 direccion   | Privada Norte Zelaya 134 328                  
 telefono    | +41(7)6995893084                              
 estado      | AGS                                           
-RECORD 1----------------------------------------------------
 sucursal_id | C-41tT                                        
 sucursal    | SON-aHW                                       
 direccion   | Callejon Gutierrez 258 636                    
 telefono    | (553)445-2016x733                             
 estado      | SON                                           
-RECORD 2----------------------------------------------------
 sucursal_id | C-59hj                                        
 sucursal    | SLP-lxZ                                       
 direccion   | Cerrada Sur Sanabria 605 805                  
 telefon

### Almacenamiento

In [9]:
nombre_destino = 'ecommerce/curated/detalle_sucursal_estado.parquet'

In [10]:
df_estado.write.mode('overwrite').parquet(dir_archivo+nombre_destino)

#### Adicional

In [11]:
df_filtrado = df_estado.limit(10)

In [12]:
df_filtrado.show(3)

+-----------+--------+--------------------+-------------------+------+
|sucursal_id|sucursal|           direccion|           telefono|estado|
+-----------+--------+--------------------+-------------------+------+
|     C-72pY| AGS-SVu|Privada Norte Zel...|   +41(7)6995893084|   AGS|
|     C-41tT| SON-aHW|Callejon Gutierre...|  (553)445-2016x733|   SON|
|     C-59hj| SLP-lxZ|Cerrada Sur Sanab...|(140)904-7730x56861|   SLP|
+-----------+--------+--------------------+-------------------+------+
only showing top 3 rows



In [13]:
df_pandas = df_filtrado.toPandas()
nombre_csv = "output/detalle_sucursal_estado.csv"
df_pandas.to_csv(nombre_csv, index=False)

### Unpersist

In [14]:
df_estado.unpersist()

DataFrame[sucursal_id: string, sucursal: string, direccion: string, telefono: string, estado: string]